<a href="https://colab.research.google.com/github/viniciusrpb/cic0269_natural_language_processing/blob/main/lectures/aula_named_entity_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aplicação: Reconhecimento de Entidades Nomeadas

https://www.tensorflow.org/datasets/catalog/conll2003


CoNLL 2003
This dataset includes 1,393 English and 909 German news articles. The English-language corpus is free, but the German corpus comes at $75, unfortunately. This is the only corpus that costs something in this post. To build the English-language corpus you need the RCV1 Reuters corpus. You will obtain access a couple days after submitting the organisational and individual agreement at no charge.

Entities are annotated with LOC (location), ORG (organisation), PER (person) and MISC (miscellaneous). 

In [1]:
!pip install -U tensorflow-datasets
!pip install keras
!pip install tensorflow
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 24.0 MB/s 
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.6.0
    Uninstalling tensorflow-datasets-4.6.0:
      Successfully uninstalled tensorflow-datasets-4.6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 13.6 MB/s 


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
import pandas as pd
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential,Model
from keras.layers import Dense,Activation,Embedding,Dropout,LSTM,Bidirectional,TimeDistributed,Input,SpatialDropout1D
from keras.utils.np_utils import to_categorical
from sklearn.metrics import classification_report
import tensorflow_addons as tfa

In [3]:
def preprocessDataFrame(df):

    dic = {}
    dic['tokens'] = []

    for text in df['tokens']:
        tokens = []
        for x in text:
            tokens.append(x.decode('utf-8'))
        l = " ".join(tokens)
        dic['tokens'].append(l.split())
        
    res_df = pd.DataFrame.from_dict(dic)
    res_df['ner'] = df['ner']
    return res_df

In [4]:
ds_train = tfds.load('conll2003', split='train', shuffle_files=True)
ds_valid = tfds.load('conll2003', split='dev', shuffle_files=False)
ds_test = tfds.load('conll2003', split='test', shuffle_files=False)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/14042 [00:00<?, ? examples/s]

...

Generating dev examples...:   0%|          | 0/3251 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/conll2003/conll2022/1.0.0.incompleteSX53U0/conll2003-dev.tfrecord*...:   0…

Generating test examples...:   0%|          | 0/3454 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/conll2003/conll2022/1.0.0.incompleteSX53U0/conll2003-test.tfrecord*...:   …

Dataset conll2003 downloaded and prepared to /root/tensorflow_datasets/conll2003/conll2022/1.0.0. Subsequent calls will reuse this data.


In [5]:
df_train = preprocessDataFrame(tfds.as_dataframe(ds_train))
df_valid = preprocessDataFrame(tfds.as_dataframe(ds_valid))
df_test = preprocessDataFrame(tfds.as_dataframe(ds_test))

In [6]:
df_train

,tokens,ner
0,"["", If, they, 're, saying, at, least, 20, perc...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[Lauck, 's, lawyer, vowed, he, would, appeal, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[Thailand, 's, powerful, military, thinks, the...","[5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, ..."
3,"[A, forensic, scientist, who, examined, the, s...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,"[Werder, Bremen, 3, 0, 1, 2, 4, 6, 1]","[3, 4, 0, 0, 0, 0, 0, 0, 0]"
...,...,...
14037,"["", He, was, not, involved, ...]","[0, 0, 0, 0, 0, 0]"
14038,"["", It, goes, without, saying, that, we, 're, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
14039,"[Bowling, :]","[0, 0]"
14040,"[National, League]","[7, 8]"


In [7]:
def label2int():
    iob_labels = ["B", "I"]
    ner_labels = ["PER", "ORG", "LOC", "MISC"]
    all_labels = [(label1, label2) for label2 in ner_labels for label1 in iob_labels]
    all_labels = ["-".join([a, b]) for a, b in all_labels]
    dic = dict(zip(range(1, len(all_labels) + 1), all_labels))
    dic[0] = 'O'
    return dic 

In [8]:
int2tag = label2int()

int2tag

{1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC',
 0: 'O'}

In [9]:
tag2int = {}
for key in int2tag:
    value = int2tag[key]
    tag2int[value] = key
print(tag2int)

num_labels = len(tag2int)

{'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8, 'O': 0}


Cria o vocabulário

In [10]:
word2index = {}
word2index['<OOV>'] = 0
word2index['<BEG>'] = 1
word2index['<END>'] = 2

ind = 3
for text in df_train['tokens']:
    for word in text:
        if word not in word2index:
            word2index[word] = ind
            ind+=1

index2word = {}
for key in word2index:
    value = word2index[key]
    index2word[value] = key

vocab_size = len(word2index)

Pré-processamento

[198380, 6535, 4351, 6218, 3632, 7030, 1107, 3414, 1038]

In [11]:
def rebuild_datasets(df,labels):

    data = []
    novel_labels = []

    idx = 0
    for text in df:

        sub = 0

        data.append(['<BEG>'])
        novel_labels.append(0)

        for i in range(0,len(text)):
            lista = ['<BEG>']+text[:i]

            data.append(lista)
            if i < len(text)-1:
                novel_labels.append(labels[idx][i])
            else:
                novel_labels.append(0)

        idx+=1

        data.append(lista+['<END>'])
        novel_labels.append(0)

    return data,novel_labels

In [12]:
x_train,y_train_int = rebuild_datasets(df_train['tokens'],df_train['ner'])
x_valid,y_valid_int = rebuild_datasets(df_valid['tokens'],df_valid['ner'])
x_test,y_test_int = rebuild_datasets(df_test['tokens'],df_test['ner'])

In [13]:
hist = [0]*num_labels
for i in y_train_int:
    hist[i]+=1

print(hist)

[198380, 6535, 4351, 6218, 3632, 7030, 1107, 3414, 1038]


Verificando se está tudo certo

In [14]:
for i,word in enumerate(x_train[33]):
    print(f'Word: {word} text label: {int2tag[y_train_int[i]]} int label: {y_train_int[i]} ')

Word: <BEG> text label: O int label: 0 
Word: " text label: O int label: 0 
Word: If text label: O int label: 0 
Word: they text label: O int label: 0 
Word: 're text label: O int label: 0 
Word: saying text label: O int label: 0 
Word: at text label: O int label: 0 
Word: least text label: O int label: 0 
Word: 20 text label: O int label: 0 
Word: percent text label: O int label: 0 
Word: , text label: O int label: 0 
Word: then text label: O int label: 0 
Word: their text label: O int label: 0 
Word: internal text label: O int label: 0 
Word: forecasts text label: O int label: 0 
Word: are text label: O int label: 0 
Word: probably text label: O int label: 0 
Word: saying text label: O int label: 0 
Word: 25 text label: O int label: 0 
Word: or text label: O int label: 0 
Word: 30 text label: O int label: 0 
Word: percent text label: O int label: 0 
Word: , text label: O int label: 0 
Word: " text label: O int label: 0 
Word: said text label: O int label: 0 
Word: one text label: O i

In [15]:
max_length = max([len(sentence) for sentence in x_train])
max_length

114

In [16]:
def text2sequences(data,vocab):
    corpus_int = []
    for instance in data:
        instance_int = []
        for term in instance:
            if term in vocab:
                instance_int.append(vocab[term])
            else:
                instance_int.append(vocab['<OOV>'])
        corpus_int.append(instance_int)
    
    return corpus_int

In [17]:
train_sequences = text2sequences(x_train,word2index)
valid_sequences = text2sequences(x_valid,word2index)
test_sequences = text2sequences(x_test,word2index)

In [18]:
trunc_type = 'post'
padding_type = 'pre'

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
valid_padded = pad_sequences(valid_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [19]:
y_train = to_categorical(y_train_int)
y_valid = to_categorical(y_valid_int)
y_test = to_categorical(y_test_int)

In [20]:
vocab_size = len(word2index)

In [21]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size+3,output_dim=64,input_length=max_length))
model.add(Bidirectional(LSTM(100,activation='relu')))
model.add(Dropout(0.5))
model.add(Dense(num_labels,activation='softmax'))

In [22]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 114, 64)           1512256   
                                                                 
 bidirectional (Bidirectiona  (None, 200)              132000    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 9)                 1809      
                                                                 
Total params: 1,646,065
Trainable params: 1,646,065
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
f1scoretfa = tfa.metrics.F1Score(num_classes=num_labels, average='macro',threshold=0.5)

sgd = SGD(learning_rate = 0.001)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy',f1scoretfa])
history_fine = model.fit(train_padded,y_train,batch_size=64,validation_data=(valid_padded,y_valid),epochs=15)

Epoch 1/15
3621/3621 [==============================] - 1204s 332ms/step - loss: 287.5390 - accuracy: 0.8607 - f1_score: 0.1402 - val_loss: 0.5247 - val_accuracy: 0.8728 - val_f1_score: 0.2153
Epoch 2/15
3621/3621 [==============================] - 1194s 330ms/step - loss: 0.4586 - accuracy: 0.8797 - f1_score: 0.2981 - val_loss: 0.4617 - val_accuracy: 0.8835 - val_f1_score: 0.3357
Epoch 3/15
3621/3621 [==============================] - 1188s 328ms/step - loss: 16069863.0000 - accuracy: 0.8884 - f1_score: 0.3856 - val_loss: 0.4742 - val_accuracy: 0.8858 - val_f1_score: 0.3642
Epoch 4/15
3621/3621 [==============================] - 1186s 328ms/step - loss: 0.3655 - accuracy: 0.8958 - f1_score: 0.4519 - val_loss: 0.4881 - val_accuracy: 0.8868 - val_f1_score: 0.3904
Epoch 5/15
3621/3621 [==============================] - 1193s 329ms/step - loss: 0.3398 - accuracy: 0.9009 - f1_score: 0.5034 - val_loss: 0.5018 - val_accuracy: 0.8865 - val_f1_score: 0.4241
Epoch 6/15
2595/3621 [==============

In [ ]:
plt.plot(history_fine.history['loss'])
plt.plot(history_fine.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_fine.history['f1_score'])
plt.plot(history_fine.history['val_f1_score'])
plt.title('model f1 score')
plt.ylabel('f1score')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

Teste

In [ ]:
y_prob = model.predict(test_padded)
y_pred = np.argmax(y_prob,axis=1)

In [ ]:
print(classification_report(y_test_int,y_pred))

Deployment

In [ ]:
sentenca_teste = ["Mayara is living in Spain which its land urges for coffee".split()]
sentenca_int = text2sequences(sentenca_teste,word2index)
sentenca_padded = pad_sequences(sentenca_int, maxlen=max_length, padding=padding_type, truncating=trunc_type)

y_prob = model.predict(sentenca_padded)
y_pred = np.argmax(y_prob,axis=1)

for idx,word in enumerate(sentenca_teste):
    print(f'{word} is label {int2tag[y_pred[idx]]}')

In [ ]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

displacy.serve(doc, style="ent")

## Estratégia 2

Utilizando a (TimeDistributed Layer)[https://keras.io/api/layers/recurrent_layers/time_distributed/] e (Spatial Dropout Layer)[https://keras.io/api/layers/regularization_layers/spatial_dropout1d/].

In [ ]:
max_length = max([len(sentence) for sentence in df_train['tokens']])
max_length

In [ ]:
train_sequences = text2sequences(df_train['tokens'],word2index)
valid_sequences = text2sequences(df_valid['tokens'],word2index)
test_sequences = text2sequences(df_test['tokens'],word2index)

In [ ]:
trunc_type = 'post'
padding_type = 'pre'

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
valid_padded = pad_sequences(valid_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
y_train_int = pad_sequences(df_train['ner'], maxlen=max_length, padding=padding_type, truncating=trunc_type)
y_valid_int = pad_sequences(df_valid['ner'], maxlen=max_length, padding=padding_type, truncating=trunc_type)
y_test_int = pad_sequences(df_test['ner'], maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
input_word = Input(shape=(max_length,))
model = Embedding(input_dim=vocab_size+3, output_dim=50)(input_word)
model = SpatialDropout1D(0.1)(model)
model = Bidirectional(LSTM(units=100, return_sequences=True,recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(num_labels, activation="softmax"))(model)
model = Model(input_word, out)
model.summary()

In [ ]:
f1scoretfa = tfa.metrics.F1Score(num_classes=num_labels, average='macro',threshold=0.5)

model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
history = model.fit(
    x=train_padded,
    y=y_train_int,
    validation_data=(valid_padded,y_valid_int),
    batch_size=64, 
    epochs=5,
    verbose=1
)

In [ ]:
!pip install seqeval

In [ ]:
from seqeval.metrics import f1_score, classification_report

In [ ]:
y_prob = model.predict(test_padded)
y_pred = []
for sentence in y_prob:
    y_pred.append(np.argmax(sentence,axis=1))
y_pred = np.array(y_pred)

In [ ]:
y_pred.shape

In [ ]:
np.array(y_test_int).shape

In [ ]:
for sentence in y_test_int:
    

In [ ]:
from sklearn.metrics import f1_score

f1=[]
for i in range(0,len(y_test_int)):
    f1.append(f1_score(y_test_int[i], y_pred[i], average='macro'))

print(np.mean(np.array(f1)))